In [ ]:
import s3fs
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import boto3
from pathlib import Path
import os
from tqdm import tqdm
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [19]:
email = input("Enter your email ENSAE: ")
name = input("Enter your username Onyxia: ")

subprocess.run(["git", "config", "--global", "user.email", email])
subprocess.run(["git", "config", "--global", "user.name", name])

print(f"Git configured with email : {email} and username : {name}")

Git configured with email : leo.leroy@ensae.fr and username : leoacpr


In [20]:
aws_access_key_id = input("Enter your AWS_ACCESS_KEY_ID: ")
aws_secret_access_key = input("Enter your AWS_SECRET_ACCESS_KEY: ")
aws_session_token = input("Enter your AWS_SESSION_TOKEN: ")

# Environment variables
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
os.environ["AWS_SESSION_TOKEN"] = aws_session_token

print("AWS keys configured as environment variables.")

AWS keys configured as environment variables.


In [25]:
# Connexion to  MinIO S3 Onyxia
s3 = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key=os.getenv("AWS_ACCESS_KEY_ID"),
    secret=os.getenv("AWS_SECRET_ACCESS_KEY"),
    token=os.getenv("AWS_SESSION_TOKEN")
)
print(len(s3.ls("projet-statapp-segmedic/metrics_results")))

61


In [42]:
import s3fs
from pathlib import Path

def download_s3_folders():
    base_local_path = Path("/tmp/metrics_results")
    base_local_path.mkdir(parents=True, exist_ok=True)

    s3_base_path = "projet-statapp-segmedic/metrics_results"

    print(f"\nTéléchargement des dossiers depuis {s3_base_path}...")

    # List all folders in the s3 base path
    folders = s3.ls(s3_base_path)

    for folder in folders:
        # Extract the folder name
        folder_name = Path(folder).name
        local_folder_path = base_local_path / folder_name
        local_folder_path.mkdir(parents=True, exist_ok=True)

        # List all files in the folder
        files = s3.ls(folder)

        for file in files:
            # Extract the file name
            file_name = Path(file).name
            local_file_path = local_folder_path / file_name

            try:
                print(f"→ Téléchargement de {file} → {local_file_path}")
                s3.get(file, str(local_file_path))
            except Exception as e:
                print(f"Erreur lors du téléchargement de {file}: {e}")

    print("\n✅ Téléchargement terminé.")

download_s3_folders()



Téléchargement des dossiers depuis projet-statapp-segmedic/metrics_results...
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL003/metrics.csv → /tmp/metrics_results/UKCHLL003/metrics.csv
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL005/metrics.csv → /tmp/metrics_results/UKCHLL005/metrics.csv
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL007/metrics.csv → /tmp/metrics_results/UKCHLL007/metrics.csv
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL008/metrics.csv → /tmp/metrics_results/UKCHLL008/metrics.csv
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL010/.keep → /tmp/metrics_results/UKCHLL010/.keep
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL010/metrics.csv → /tmp/metrics_results/UKCHLL010/metrics.csv
→ Téléchargement de projet-statapp-segmedic/metrics_results/UKCHLL013/metrics.csv → /tmp/metrics_results/UKCHLL013/metrics.csv
→ Téléchargement de projet-statapp-segmedic/

In [66]:
import os
import pandas as pd
from pathlib import Path

def concatenate_csv_files():
    base_path = Path("/tmp/metrics_results")
    output_file = base_path / "concatenated_results.csv"

    # Initialize an empty DataFrame to store the concatenated data
    concatenated_df = pd.DataFrame()

    # Iterate over each folder in the base directory
    for folder in base_path.iterdir():
        if folder.is_dir():
            #print(f"Traitement du dossier: {folder.name}")

            # Iterate over each file in the folder
            for file in folder.iterdir():
                if file.is_file() and file.suffix == ".csv":
                    #print(f"→ Lecture du fichier: {file.name}")

                    try:
                        # Read the CSV file
                        df = pd.read_csv(file)

                        # Add a column to identify the source folder
                        df["source_folder"] = folder.name
                        #print(df.head())

                        # Concatenate the data
                        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
                    except Exception as e:
                        print(f"Erreur lors de la lecture de {file.name}: {e}")

    return concatenated_df

df = concatenate_csv_files()
print(df.shape)
df.to_csv("/home/onyxia/work/statapp_2025_curvas/data_metrics.csv")


(789, 36)


In [67]:
print(df.head())

  Patient_ID                        CT  DICE_panc  DICE_kidn  DICE_livr  \
0        110  anno1_init112233_foldall   0.907189   0.968869   0.970781   
1        110  anno1_init445566_foldall   0.910028   0.971575   0.974230   
2        110  anno1_init778899_foldall   0.886513   0.970200   0.970827   
3        110  anno2_init112233_foldall   0.919737   0.980913   0.980391   
4        110  anno2_init445566_foldall   0.909938   0.980682   0.980590   

   CONF_panc  CONF_kidn  CONF_livr  Entropy_GT  Entropy_Pred  ...  EAURC_kidn  \
0   0.967983   0.982864   0.989874    0.265031      0.258090  ...    0.078258   
1   0.963863   0.983514   0.991349    0.265031      0.258891  ...    0.066482   
2   0.964050   0.983202   0.990331    0.265031      0.257999  ...    0.101868   
3   0.982944   0.992676   0.994331    0.265031      0.265384  ...    0.060995   
4   0.983300   0.993238   0.994168    0.265031      0.264757  ...    0.064607   

   EAURC_livr    CRPS_panc     CRPS_kidn     CRPS_livr  NCC_GT

In [68]:
df.CT.unique()

array(['anno1_init112233_foldall', 'anno1_init445566_foldall',
       'anno1_init778899_foldall', 'anno2_init112233_foldall',
       'anno2_init445566_foldall', 'anno2_init778899_foldall',
       'anno3_init112233_foldall', 'anno3_init445566_foldall',
       'anno3_init778899_foldall', 'ensemble_all', 'ensemble_anno1',
       'ensemble_anno2', 'ensemble_anno3'], dtype=object)

Statistique d'agrégation globale sur toutes les métriques

In [ ]:
# Création d'une nouvelle colonne pour regrouper les modèles
df['model_group'] = 'individual'

# Identifier les lignes contenant "foldall"
df.loc[df['CT'].str.contains("foldall"), 'model_group'] = 'individual_anno'

# Identifier les modèles "ensemble" qui ne sont pas "foldall"
ensemble_mask = df['CT'].str.contains("ensemble") 
df.loc[ensemble_mask, 'model_group'] = df.loc[ensemble_mask, 'CT']


# Vérifions la distribution des groupes
df['model_group'].value_counts()

# Liste des colonnes d'incertitude à analyser
uncertainty_columns = [
    "Entropy_Pred",
    "CONF_panc", "CONF_kidn", "CONF_livr",
    "ECE_1", "ECE_2", "ECE_3",
    "ACE_1", "ACE_2", "ACE_3",
    "CRPS_panc", "CRPS_kidn", "CRPS_livr",
    "EAURC_panc", "EAURC_kidn", "EAURC_livr",
    "AURC_panc", "AURC_kidn", "AURC_livr",
    "AUROC_panc", "AUROC_kidn", "AUROC_livr"
]

# Moyenne et écart-type par groupe
grouped_stats = df[df['model_group'].str.contains("ensemble")].groupby("model_group")[uncertainty_columns].agg(['mean', 'std', 'median'])

# Arrondir pour la lisibilité
grouped_stats = grouped_stats.round(4)

grouped_stats.T  # Transposition pour affichage vertical (par métrique)


model_group          ensemble_all  ensemble_anno1  ensemble_anno2  \
Entropy_Pred mean          0.2714          0.2673          0.2738   
             std           0.0393          0.0387          0.0394   
             median        0.2655          0.2616          0.2678   
CONF_panc    mean          0.9348          0.9326          0.9425   
             std           0.0804          0.0813          0.0807   
...                           ...             ...             ...   
AUROC_kidn   std           0.0678          0.0591          0.0637   
             median        0.7320          0.8134          0.7529   
AUROC_livr   mean          0.8857          0.8910          0.8866   
             std           0.0647          0.0782          0.0568   
             median        0.8947          0.9013          0.8943   

model_group          ensemble_anno3  
Entropy_Pred mean            0.2733  
             std             0.0399  
             median          0.2672  
CONF_panc    mean            0.9292  
             std             0.0809  
...                             ...  
AUROC_kidn   std             0.0725  
             median          0.6771  
AUROC_livr   mean            0.8925  
             std             0.0634  
             median          0.8970  

[66 rows x 4 columns]

Comparaison avec Benchmark (attention la stratégie d'agrégation n'est peut-être pas la bonne)


In [98]:
dice_cols = [col for col in df.columns if col.startswith("DICE_")]
conf_cols = [col for col in df.columns if col.startswith("CONF_")]
ece_cols = [col for col in df.columns if col.startswith("ECE_")]
crps_cols = [col for col in df.columns if col.startswith("CRPS_")]

# regrouper les scores selon les organes en prenant la moyenne
df["DSC"] = df[dice_cols].mean(axis=1)
df["Confiance"] = df[conf_cols].mean(axis=1)
df["ECE"] = df[ece_cols].mean(axis=1)
df["CRPS"] = df[crps_cols].mean(axis=1)

categories = {
    "foldall (1 anno)": df[df["CT"].str.contains("foldall")],
    "ensemble_anno1": df[df["CT"].str.contains("ensemble_anno1")],
    "ensemble_anno2": df[df["CT"].str.contains("ensemble_anno2")],
    "ensemble_anno3": df[df["CT"].str.contains("ensemble_anno3")],
    "ensemble_all": df[df["CT"].str.contains("ensemble_all")],
}

results = []
for label, group in categories.items():
    median_dsc = group["DSC"].median()
    median_conf = group["Confiance"].median()
    median_ece = group["ECE"].median()
    median_crps = group["CRPS"].median()/1000 #for cm^3
    results.append([label, median_dsc, median_conf, median_ece, median_crps])

benchmark_comp = pd.DataFrame(
    results,
    columns=["Catégorie", "DSC médian", "Confiance médiane", "ECE médian", "CRPS médian"]
)

print(benchmark_comp)




          Catégorie  DSC médian  Confiance médiane  ECE médian  CRPS médian
0  foldall (1 anno)    0.939041           0.981437    0.002729    13.389282
1    ensemble_anno1    0.935027           0.978734    0.002737    14.423758
2    ensemble_anno2    0.949254           0.984980    0.002697    11.211157
3    ensemble_anno3    0.940275           0.980048    0.002211    13.335825
4      ensemble_all    0.946949           0.980680    0.001829    10.275960


Violin box

In [1]:
def violin(metric): 
    df_plot = df.copy()
    # Fonction pour assigner les groupes avec couleurs spécifiques
    def assign_group(ct):
        if ct == 'ensemble_all':
            return 'Ensemble général'
        elif ct.startswith('ensemble_anno'):
            return 'Ensemble par annotateurs'
        elif 'foldall' in ct:
            return 'Modèles individuels'

    df_plot["Group"] = df_plot["CT"].apply(assign_group)

    # Filtrer pour ne garder que les 3 groupes souhaités
    df_plot = df_plot[df_plot["Group"].isin(['Modèles individuels', 'Ensemble par annotateurs', 'Ensemble général'])]

    df_plot = df_plot.melt(
        id_vars=["CT", "Group"],
        value_vars=[f"{metric}_panc", f"{metric}_kidn", f"{metric}_livr"],
        var_name="Organ",
        value_name="value"
    )

    # Palette personnalisée pour les couleurs
    color_map = {
        "Modèles individuels": "blue",
        "Ensemble par annotateurs": "red",
        "Ensemble général": "green",
    }

    fig = px.violin(
        df_plot,
        x="Group",          # Regrouper par groupe (3 violons)
        y="value",          # dice for all organs
        color="Group",
        category_orders={"Group": ["Modèles individuels", "Ensemble par annotateurs", "Ensemble général"]},
        color_discrete_map=color_map,
        box=True,
        points="all"
    )

    fig.update_layout(
        title=f"{metric} par catégorie de modèle",
        xaxis_title="Catégorie de modèle",
        yaxis_title="metric",
        showlegend=False  # Optionnel, la couleur est explicite par axe x
    )

    fig.show()


#violin("DICE")
#doesnt work with ECE bc ECE is global, not organ-specific


ECE violin plot

In [2]:
metric = 'ECE'
df_plot = df.copy()

# Fonction pour assigner les groupes avec couleurs spécifiques
def assign_group(ct):
    if ct == 'ensemble_all':
        return 'Ensemble général'
    elif ct.startswith('ensemble_anno'):
        return 'Ensemble par annotateurs'
    elif 'foldall' in ct:
        return 'Modèles individuels'

df_plot["Group"] = df_plot["CT"].apply(assign_group)

# Filtrer pour ne garder que les 3 groupes souhaités
df_plot = df_plot[df_plot["Group"].isin(['Modèles individuels', 'Ensemble par annotateurs', 'Ensemble général'])]

# Palette personnalisée pour les couleurs
color_map = {
    "Modèles individuels": "blue",
    "Ensemble par annotateurs": "red",
    "Ensemble général": "green",
}

fig = px.violin(
    df_plot,
    x="Group",          # Regrouper par groupe (3 violons)
    y="ECE",          # ECE for all organs
    color="Group",
    category_orders={"Group": ["Modèles individuels", "Ensemble par annotateurs", "Ensemble général"]},
    color_discrete_map=color_map,
    box=True,
    points="all"
)

fig.update_layout(
    title=f"{metric} par catégorie de modèle",
    xaxis_title="Catégorie de modèle",
    yaxis_title=metric,
    showlegend=False  # Optionnel, la couleur est explicite par axe x
)

#fig.show()


NameError: name 'df' is not defined

Graphiques analyse métriques 2 à 2

Analyse :
- "Qualité ↔ Incertitude": ("DICE_", "entropy_pred")
- "Individuel ↔ Ensemble": ("DICE_", "DICE_ensemble_mean") --> déjà fait avec violin plot
- "Incertitude ↔ Ambiguïté": ("entropy_pred", "ncc_mean")
- "Qualité ↔ Ambiguïté": ("DICE_", "ncc_mean")



In [117]:
df.columns

Index(['Patient_ID', 'CT', 'DICE_panc', 'DICE_kidn', 'DICE_livr', 'CONF_panc',
       'CONF_kidn', 'CONF_livr', 'Entropy_GT', 'Entropy_Pred', 'Hausdorff_1',
       'Hausdorff_2', 'Hausdorff_3', 'ECE_1', 'ECE_2', 'ECE_3', 'ACE_1',
       'ACE_2', 'ACE_3', 'AUROC_panc', 'AUROC_kidn', 'AUROC_livr', 'AURC_panc',
       'AURC_kidn', 'AURC_livr', 'EAURC_panc', 'EAURC_kidn', 'EAURC_livr',
       'CRPS_panc', 'CRPS_kidn', 'CRPS_livr', 'NCC_GT1-2', 'NCC_GT1-3',
       'NCC_GT2-3', 'NCC_mean', 'source_folder', 'model_group', 'DSC',
       'Confiance', 'ECE', 'CRPS'],
      dtype='object')

In [3]:
#Qualité et Incertitude

organ_colors = {
    "panc": "red",
    "kidn": "green",
    "livr": "blue"
}

df_melted = pd.DataFrame()

for organ, color in organ_colors.items():
    temp_df = df[["CT", f"DICE_{organ}", f"Entropy_Pred"]].copy()
    temp_df.columns = ["CT", "Quality", "Uncertainty"]
    temp_df["Organ"] = organ
    temp_df["Color"] = color
    df_melted = pd.concat([df_melted, temp_df], ignore_index=True)

# Tracer le scatter
fig = px.scatter(
    df_melted,
    x="Quality",
    y="Uncertainty",
    color="Organ",
    color_discrete_map={"panc": "red", "kidn": "green", "livr": "blue"},
    hover_data=["CT"]
)

fig.update_layout(
    title="Qualité ↔ Incertitude : DICE vs Entropie prédictive",
    xaxis_title="DICE",
    yaxis_title="Entropie prédictive"
)
#fig.show()


NameError: name 'pd' is not defined

In [4]:
# Incertitude et Ambigüité

# Organe → couleur
organ_colors = {
    "panc": "red",
    "kidn": "green",
    "livr": "blue"
}

fig = px.scatter(
    df,
    x="Entropy_Pred",
    y="NCC_mean",
    hover_data=["CT"],
    color_discrete_sequence=["purple"]  # Couleur unique comme dans le précédent
)

fig.update_layout(
    title="Incertitude ↔ Ambiguïté : Entropie prédictive vs NCC",
    xaxis_title="Entropie prédictive",
    yaxis_title="NCC",
)
#fig.show()


NameError: name 'px' is not defined

In [5]:
#Qualité et Ambiguïté"

organ_colors = {
    "panc": "red",
    "kidn": "green",
    "livr": "blue"
}

df_melted = pd.DataFrame()

for organ, color in organ_colors.items():
    temp_df = df[["CT", f"DICE_{organ}", f"NCC_mean"]].copy()
    temp_df.columns = ["CT", "Quality", "Uncertainty"]
    temp_df["Organ"] = organ
    temp_df["Color"] = color
    df_melted = pd.concat([df_melted, temp_df], ignore_index=True)

# Tracer le scatter
fig = px.scatter(
    df_melted,
    x="Quality",
    y="Uncertainty",
    color="Organ",
    color_discrete_map={"panc": "red", "kidn": "green", "livr": "blue"},
    hover_data=["CT"]
)

fig.update_layout(
    title="Qualité ↔ Ambiguïté : DICE vs NCC",
    xaxis_title="DICE",
    yaxis_title="NCC"
)
#fig.show()


NameError: name 'pd' is not defined